# CEPEA - Merge RSS e Feed

É um _script_ para juntar as duas bases em uma única.

Será usado também uma única fez para fazer a importação inicial.

In [2]:
import os
import pandas as pd
import time

## 1. Bases

Carrega as bases do Feed e do RSS.

In [4]:
# Construir o caminho completo para o arquivo CSV
base_dir = os.getcwd()  # Diretório atual

# Carregar o DataFrame de Feed a partir do arquivo CSV
df_feed = pd.read_csv(os.path.join(base_dir, '2024-08-11 cepea_feed_list.csv'), encoding='utf-8-sig')

# Carregar o DataFrame de RSS a partir do arquivo CSV
df_rss = pd.read_csv(os.path.join(base_dir, '2024-08-11 cepea_rss.csv'), encoding='utf-8-sig')

## 2. Limpeza

Remove as linhas desnecessárias que estão em duplicatas com a RSS e Feed.

In [6]:
# Removendo as duplicatas. NÃO TEM COMO FAZER AUTOMÁTICO AQUI.
df_feed = df_feed.iloc[7:]

### 2.1. Conversão de Data

In [8]:
# Supondo que 'df_feed' seja o seu DataFrame
# Resetar os índices para evitar problemas com cópias e vistas
df_feed = df_feed.reset_index(drop=True)

# Converter todos os valores da coluna "Data" para strings
df_feed['Data'] = df_feed['Data'].astype(str)

# Remover o hífen e espaços extras do final das strings na coluna "Data"
df_feed['Data'] = df_feed['Data'].str.replace(' -', '', regex=False)

# Converter as datas para o formato datetime
df_feed['Data'] = pd.to_datetime(df_feed['Data'], format='%d/%m/%Y', errors='coerce')

# Formatar as datas no padrão 'YYYY-MM-DD'
df_feed['Data'] = df_feed['Data'].dt.strftime('%Y-%m-%d')

In [9]:
# Supondo que 'df_rss' seja o seu DataFrame
# Converter as strings de data para objetos datetime
df_rss['Data'] = pd.to_datetime(df_rss['Data'], format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')

# Extrair apenas a data no formato 'YYYY-MM-DD'
df_rss['Data'] = df_rss['Data'].dt.strftime('%Y-%m-%d')

## 3. Merge

In [11]:
df_merge = pd.concat([df_rss, df_feed], axis=0)

In [12]:
# Converter todas as datas para datetime
df_merge['Data'] = pd.to_datetime(df_merge['Data'], errors='coerce')  # 'errors="coerce"' trata entradas inválidas

# Formatar as datas para o formato 'YYYY-MM-DD'
df_merge['Data'] = df_merge['Data'].dt.strftime('%Y-%m-%d')

In [13]:
## 4. Visualização

In [14]:
# Exibir o DataFrame com a coluna Data uniformizada
df_merge.head()

,Data,Produto,Título,URL
0,2024-08-05,AÇÚCAR/CEPEA,"Com baixa liquidez, preços têm leve queda",https://www.cepea.esalq.usp.br/br/diarias-de-m...
1,2024-08-05,ETANOL/CEPEA,"Vendas ultrapassam 100 milhões de litros, e hi...",https://www.cepea.esalq.usp.br/br/diarias-de-m...
2,2024-07-29,ETANOL/CEPEA,Indicadores caem 3% em SP,https://www.cepea.esalq.usp.br/br/diarias-de-m...
3,2024-07-29,AÇÚCAR/CEPEA,Vendas internas voltam a remunerar mais que ex...,https://www.cepea.esalq.usp.br/br/diarias-de-m...
4,2024-07-22,AÇÚCAR/CEPEA,Preços sobem mesmo com baixa liquidez,https://www.cepea.esalq.usp.br/br/diarias-de-m...


In [15]:
df_merge.tail()

,Data,Produto,Título,URL
149,2023-01-24,ETANOL/CEPEA,Cotações seguem em queda por mais uma semana -,https://www.cepea.esalq.usp.br/br/diarias-de-m...
150,2023-01-17,AÇÚCAR/CEPEA,"Baixas nos preços prevalecem, apesar de entres...",https://www.cepea.esalq.usp.br/br/diarias-de-m...
151,2023-01-17,ETANOL/CEPEA,Indicadores seguem em queda -,https://www.cepea.esalq.usp.br/br/diarias-de-m...
152,2023-01-10,AÇÚCAR/CEPEA,"Com vendas enfraquecidas, preços recuam -",https://www.cepea.esalq.usp.br/br/diarias-de-m...
153,2023-01-10,ETANOL/CEPEA,Ano começa com baixa liquidez -,https://www.cepea.esalq.usp.br/br/diarias-de-m...


## 4. Exportação

In [17]:
# Exporta na pasta 01-data
df_merge.to_csv(time.strftime('%Y-%m-%d') + ' ' + 'cepea_feed_rss_merge.csv', index=False, encoding='utf-8-sig')